# 1. Liste des élus romands

In [1]:
import requests
import pandas as pd

In [2]:
members_response = requests.get('http://ws-old.parlament.ch/councillors/basicdetails?format=json&lang=fr', headers={'Accept': 'application/json'})
members = members_response.json()

In [3]:
# liste des elus
df = pd.DataFrame(members)
df.shape

(245, 16)

In [4]:
df.to_csv('data/councellors.csv')

In [5]:
# romands seulement
df = df[df['canton'].isin(['VD', 'GE', 'FR', 'JU', 'VS', 'NE'])].copy()
df.shape

(62, 16)

# 2. Liste des interventions

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
binary = FirefoxBinary('/Applications/Firefox_32.app/Contents/MacOS/firefox-bin')
driver = webdriver.Chrome(r'_helpers/chromedriver')
from time import sleep
import re

## On prend les Jurassiens pour tester

Fait aussi: Valais, Neuchâtel, Genève

In [8]:
df_ju = df[df['canton'] == 'JU']

In [9]:
member_speeches_url = 'https://www.parlament.ch/fr/ratsbetrieb/suche-Amtliches-bulletin#Default=%7B%22k%22:%22PdSubjectSpeakerPersonNumbers:{member_id}%22%7D'

In [10]:
def getSpeeches():
    search_group = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#Groups .ms-srch-group'))
    )
    
    if not search_group:
        return False
    
    # On parcourt l’innerHTML avec BeautifulSoup
    doc = BeautifulSoup(search_group.get_attribute('innerHTML'), 'html.parser')
    
    results = doc.select('.ms-srch-item')

    current_rows = []
    for result in results:
        link = 'https://www.parlament.ch' + result.find('a').get('href')
        label = result.find('h4').text.strip()
        no = result.find('div', {'class': 'ms-srch-item-excerpt'}).find('span').string
        title = result.find('div', {'class': 'ms-srch-item-excerpt'}).find_all('span')[1].string
        current_rows.append([no, label, title, link])
    print(len(current_rows), 'rows found.')
    return current_rows

In [11]:
for i, row in df[df['canton'].isin(['VD', 'FR'])].iterrows():
    target_id = row['id']    
    print(row['firstName'], row['lastName'], target_id)
    driver.get(member_speeches_url.format(member_id=target_id))
    rows = []
    continueScraping = True
    
    sleep(2)
    
    while continueScraping:
        scraping_result = getSpeeches()
        if scraping_result:
            rows.extend(scraping_result)
            #dfi = pd.DataFrame(rows, columns=['no', 'label', 'title', 'link'])
            last_result_year = re.match('.*\.(\d{4}).*', scraping_result[-1][1]).group(1)
            if int(last_result_year) < 2015:
                print('Before 2015, so we stop (year = {})'.format(last_result_year))
                # continueScraping = False
                break
                
            # On cherche la flèche qui charge la prochaine page
            try:
                nextPageLink = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#PageLinkNext'))
                    )
                if nextPageLink:
                    nextPageLink.click()
                    sleep(2.5)
                else:
                    print('Next page link not working?')
            except TimeoutException as ex:
                print('Last page, so we stop')
                continueScraping = False
        else:
            print('No result in this page!')
            
    df_member = pd.DataFrame(rows, columns=['no', 'label', 'title', 'href'])
    
    print('df shape:', df_member.shape)
    print('Without duplicate no:', df_member.drop_duplicates(subset=['no']).shape)
    print('Without duplicate href:', df_member.drop_duplicates(subset=['href']).shape)
    
    filepath = 'data/speech_lists/{}.csv'.format(target_id)
    print('Saving df to', filepath)
    df_member.to_csv(filepath)
    print('–––')
    sleep(3.5)

Claude Béglé 4182
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
9 rows found.
Last page, so we stop
df shape: (89, 4)
Without duplicate no: (87, 4)
Without duplicate href: (89, 4)
Saving df to data/speech_lists/4182.csv
–––
Samuel Bendahan 4213
10 rows found.
10 rows found.
10 rows found.
Last page, so we stop
df shape: (30, 4)
Without duplicate no: (20, 4)
Without duplicate href: (30, 4)
Saving df to data/speech_lists/4213.csv
–––
Frédéric Borloz 4188
10 rows found.
10 rows found.
Last page, so we stop
df shape: (20, 4)
Without duplicate no: (18, 4)
Without duplicate href: (20, 4)
Saving df to data/speech_lists/4188.csv
–––
Jacques Bourgeois 3872
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
Before 2015, so we stop (year = 2014)
df shape: (90, 4)
Without duplicate no: (75, 4)
Without duplicate href: (90, 4)
Saving df to data

Adèle Thorens Goumaz 3907
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
Before 2015, so we stop (year = 2014)
df shape: (70, 4)
Without duplicate no: (54, 4)
Without duplicate href: (70, 4)
Saving df to data/speech_lists/3907.csv
–––
Beat Vonlanthen 4205
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
4 rows found.
Last page, so we stop
df shape: (94, 4)
Without duplicate no: (84, 4)
Without duplicate href: (94, 4)
Saving df to data/speech_lists/4205.csv
–––
Laurent Wehrli 4194
10 rows found.
10 rows found.
8 rows found.
Last page, so we stop
df shape: (28, 4)
Without duplicate no: (23, 4)
Without duplicate href: (28, 4)
Saving df to data/speech_lists/4194.csv
–––


In [37]:
# Rattrapage si ça a foiré

df_member = pd.DataFrame(rows, columns=['no', 'label', 'title', 'href'])

print('df shape:', df_member.shape)
print('Without duplicate no:', df_member.drop_duplicates(subset=['no']).shape)
print('Without duplicate href:', df_member.drop_duplicates(subset=['href']).shape)

filepath = 'data/speech_lists/{}.csv'.format(target_id)
print('Saving df to', filepath)
df_member.to_csv(filepath)

df shape: (76, 4)
Without duplicate no: (68, 4)
Without duplicate href: (76, 4)
Saving df to data/speech_lists/4154.csv


# 3. Scraping des interventions (test)

Ça se passe dans Get_speeches.ipynb